In [3]:
from gru.gru_v3 import EncoderDecoder
from gru.dataset import GRUDataset_v2
from gru import example_printer
from gru.cce import CCE, ConsciousCrossEntropy
import torch
import wandb
import pandas as pd
import torch.nn as nn
from selfies_tools import SELFIESVectorizer, determine_alphabet
import random
import torch
model = EncoderDecoder

device = 'cuda'
print(device)

# Set hyperparameters
encoding_size = 512
hidden_size = 512
num_layers = 1
learn_rate = 0.0003
dropout = 0 # dropout must be equal 0 if num_layers = 1
teacher_ratio = 0.5

# Init model
model = EncoderDecoder(
    fp_size=4860,
    encoding_size=encoding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    teacher_ratio = teacher_ratio).to(device)

model.load_state_dict(torch.load('PATH'))

cuda


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [2]:
alphabet = ['[#Branch1]',
 '[#Branch2]',
 '[#C]',
 '[#N]',
 '[/C]',
 '[/N]',
 '[/O]',
 '[=Branch1]',
 '[=Branch2]',
 '[=C]',
 '[=N+1]',
 '[=N]',
 '[=O]',
 '[=Ring1]',
 '[=Ring2]',
 '[=S]',
 '[Br]',
 '[Branch1]',
 '[Branch2]',
 '[C@@H1]',
 '[C@@]',
 '[C@H1]',
 '[C@]',
 '[C]',
 '[Cl]',
 '[F]',
 '[I]',
 '[N+1]',
 '[NH1]',
 '[N]',
 '[O-1]',
 '[O]',
 '[P]',
 '[Ring1]',
 '[Ring2]',
 '[S]',
 '[\\C]',
 '[\\N]',
 '[\\O]',
 '[end]',
 '[nop]',
 '[start]']

In [3]:
data_path = './GRU_data/combined_smiles_fp.parquet'
vectorizer = SELFIESVectorizer(alphabet, pad_to_len=128)

dataset = GRUDataset_v2(data_path, vectorizer)
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print("Dataset size:", len(dataset))
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

Dataset size: 385165
Train size: 346648
Test size: 38517


In [4]:
from torch.utils.data import DataLoader
batch_size = 256
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, drop_last=True)

In [5]:
x, y = next(iter(train_loader))
x = x.to(device)
y = y.to(device)

In [185]:
out = model(x, y, teacher_forcing=False)
softmax = nn.Softmax(dim=2)
out = softmax(out)
out = out.detach().cpu().numpy()
target = y.detach().cpu().numpy()

out.shape

(256, 128, 42)

In [172]:
import selfies as sf
import rdkit.Chem as Chem

preds = []
targets = []

for n in range(256):
    selfie_out = vectorizer.devectorize(out[n], remove_special=True)
    selfie_target = vectorizer.devectorize(target[n], remove_special=True)
    smiles_out = sf.decoder(selfie_out)
    smiles_target = sf.decoder(selfie_target)
    mol_out = Chem.MolFromSmiles(smiles_out)
    mol_target = Chem.MolFromSmiles(smiles_target)
    preds.append(mol_out)
    targets.append(mol_target)

In [283]:
idx += 1
print(idx)
Chem.Draw.MolsToImage([targets[idx], preds[idx]], subImgSize=(400, 400), legends=None)

256


IndexError: list index out of range